In [ ]:
import requests as rqst
import subprocess
import ipaddress
from subprocess import Popen, PIPE
import pandas as pd
from datetime import date,timedelta
import pymongo


ip_online = []
ip_net    = ipaddress.ip_network(u'192.168.1.0/24', strict=False)
print("Analizando red...")
for ip in ip_net.hosts():
    ip = str(ip)
    toping = Popen(['ping', '-c', '1', '-W', '50', ip], stdout=PIPE)
    output = toping.communicate()[0]
    hostalive = toping.returncode
    if hostalive == 0:
        ip_online.append(ip)
    if ip == "192.168.1.50":
        break

ip_net    = ipaddress.ip_network(u'192.168.2.0/24', strict=False)
print("Analizando red...")
for ip in ip_net.hosts():
    ip = str(ip)
    toping = Popen(['ping', '-c', '1', '-W', '50', ip], stdout=PIPE)
    output = toping.communicate()[0]
    hostalive = toping.returncode
    if hostalive == 0:
        ip_online.append(ip)
    if ip == "192.168.2.50":
        break

ip_net    = ipaddress.ip_network(u'192.168.3.0/22', strict=False)
print("Analizando red...")
for ip in ip_net.hosts():
    ip = str(ip)
    toping = Popen(['ping', '-c', '1', '-W', '50', ip], stdout=PIPE)
    output = toping.communicate()[0]
    hostalive = toping.returncode
    if hostalive == 0:
        ip_online.append(ip)
    if ip == "192.168.3.10":
        break

print(len(ip_online),"host encontrados")
data_ip = []
for ip in ip_online:
    try:  
        r = rqst.get(f'http://{ip}:5600/',verify=False, timeout=3)
        if r.status_code == 200:
            data_ip.append(ip)
    except: 
        print(ip,"ERROR: ")
        
data_ip


Analizando red...
Analizando red...


In [37]:
data = []

today1    = "2019-15-01"
today2    = "2020-15-01"

for ip in data_ip:
    payload = "{\n    \"query\": [\n        \"events  = flood(query_bucket(find_bucket('aw-watcher-afk_')));\",\n        \"not_afk = flood(query_bucket(find_bucket('aw-watcher-afk_')));\",\n        \"not_afk = filter_keyvals(not_afk, 'status', ['not-afk']);\",\n        \"events  = filter_period_intersect(events, not_afk);\",\n        \"title_events = sort_by_duration(merge_events_by_keys(events, ['app', 'title']));\",\n        \"app_events   = sort_by_duration(merge_events_by_keys(title_events, ['app']));\",\n        \"cat_events   = sort_by_duration(merge_events_by_keys(events, ['$category']));\",\n        \"events = sort_by_timestamp(events);\",\n        \"app_events  = limit_events(app_events, 100);\",\n        \"title_events  = limit_events(title_events, 100);\",\n        \"duration = sum_durations(events);\",\n        \"RETURN  = {'app_events': app_events, 'title_events': title_events, 'cat_events': cat_events, 'duration': duration, 'active_events': not_afk};\"\n    ],\n    \"timeperiods\": [\n        \""+today1+"/"+today2+"\"\n    ]\n}"
    headers = {
        'Content-Type': "application/json"
    }
    r = rqst.request("POST", f'http://{ip}:5600/api/0/query/', data = payload, headers=headers)
    print(r.status_code)
    if r.status_code == 200:   
        response = r.json()
        prine(response)
                

500
500
500
500
500
500
500
500
500
500
500
500
